# 19.3 Takeaway - Sync Two Interleaving Threads

## Problem

Thread t1 prints odd numbers from 1-100

Thread t2 prints even numbers from 1-100

Write code where two threads, running concurrently, print the numbers 1-100 in order.

## Naiive Approach with Busy Locking

The implementation below involves busy locking, which is a technique where a thread waits in a loop checking a condition repeatedly until it becomes true. 

In this case, the `wait_for_odd` and `wait_for_even` methods of the OddEvenMonitor class use busy waiting to wait for the other thread to finish printing and for it to become the current thread's turn. The while loops in these methods keep the thread in a busy waiting state until the condition they are checking becomes true.

In [24]:
import threading

class CounterThread(threading.Thread):
    def __init__(self, start_num, increment, monitor, is_even):
        super().__init__()
        self.start_num = start_num
        self.increment = increment
        self.monitor = monitor
        self.is_even = is_even

    def run(self):
        for i in range(self.start_num, 101, 2):
            if not self.is_even:
                self.monitor.wait_for_odd()
            else:
                self.monitor.wait_for_even()
            print(i)
            self.monitor.increment()

class OddEvenMonitor:
    def __init__(self):
        self.turn = 0

    def wait_for_odd(self):
        while self.turn % 2 != 0:
            pass

    def wait_for_even(self):
        while self.turn % 2 == 0:
            pass

    def increment(self):
        self.turn += 1

In [25]:
monitor = OddEvenMonitor()
odd_thread = CounterThread(1, 2, monitor, False)
even_thread = CounterThread(2, 2, monitor, True)

odd_thread.start()
even_thread.start()

odd_thread.join()
even_thread.join()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


## Implementation Avoiding Busy Locking

While busy waiting can be a simple and effective way to implement synchronization in certain cases, it can also be inefficient and waste CPU time if the waiting period is long. Other synchronization techniques, such as condition variables, can be used to avoid busy waiting and allow a thread to sleep until the condition it is waiting for becomes true, saving CPU time.

Below uses such condition variables wiht

In [26]:
import threading

class OddEvenMonitor:
    # Constants to represent odd and even turn
    ODD_TURN = True
    EVEN_TURN = False
    
    def __init__(self):
        # Create a lock object to synchronize access to shared state
        self.lock = threading.Lock()
        # Create two condition objects that use the same lock
        self.odd_turn = threading.Condition(self.lock)
        self.even_turn = threading.Condition(self.lock)
        # Start with odd turn
        self.turn = self.ODD_TURN

    def wait_for_odd(self):
        # Acquire the lock and check if it is the odd turn
        with self.lock:
            while self.turn != self.ODD_TURN:
                # If not, wait for notification that it is now the odd turn
                self.odd_turn.wait()

    def wait_for_even(self):
        # Acquire the lock and check if it is the even turn
        with self.lock:
            while self.turn != self.EVEN_TURN:
                # If not, wait for notification that it is now the even turn
                self.even_turn.wait()

    def toggle_turn(self):
        # Acquire the lock and switch the turn
        with self.lock:
            self.turn ^= True
            # Notify the waiting thread of the opposite turn
            if self.turn == self.ODD_TURN:
                self.odd_turn.notify()
            else:
                self.even_turn.notify()

class OddThread(threading.Thread):
    def __init__(self, monitor):
        super().__init__()
        self.monitor = monitor
    
    def run(self):
        # Loop through odd numbers from 1 to 100
        for i in range(1, 101, 2):
            # Wait for the odd turn
            self.monitor.wait_for_odd()
            # Print the odd number
            print(i)
            # Toggle the turn to even
            self.monitor.toggle_turn()

class EvenThread(threading.Thread):
    def __init__(self, monitor):
        super().__init__()
        self.monitor = monitor
    
    def run(self):
        # Loop through even numbers from 2 to 100
        for i in range(2, 101, 2):
            # Wait for the even turn
            self.monitor.wait_for_even()
            # Print the even number
            print(i)
            # Toggle the turn to odd
            self.monitor.toggle_turn()

In [27]:
# Create an instance of the OddEvenMonitor
monitor = OddEvenMonitor()
# Create an instance of the OddThread and start it
odd_thread = OddThread(monitor)
odd_thread.start()
# Create an instance of the EvenThread and start it
even_thread = EvenThread(monitor)
even_thread.start()
# Wait for the threads to finish
odd_thread.join()
even_thread.join()


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
